In [341]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import brentq
from sklearn.linear_model import SGDRegressor



### Virtual valuation and Reserve price calculator

In [342]:
class VirtualValuationCalculator:
    def virtual_valuation(self, v, mean, stddev):
        f_cdf = norm.cdf(v, loc=mean, scale=stddev)
        f_pdf = norm.pdf(v, loc=mean, scale=stddev)
        return v - (1 - f_cdf) / f_pdf


class ReservePriceCalculator(VirtualValuationCalculator):
    def __init__(self, means, stddevs):
        self.means = means
        self.stddevs = stddevs

    def phi_inverse_0(self, mean, stddev):
        def phi(v):
            return self.virtual_valuation(v, mean, stddev)  

        v_low = mean - 3 * stddev  # Lower bound for root-finding
        v_high = mean + 3 * stddev  # Upper bound for root-finding
        reserve_price = brentq(phi, v_low, v_high)
        
        return reserve_price

    def calculate_reserve_prices(self):
        reserve_prices = []
        for mean, stddev in zip(self.means, self.stddevs):
            r_star = self.phi_inverse_0(mean, stddev)
            reserve_prices.append(r_star)
        return reserve_prices
    

### Set up participant data

In [343]:
# Example usage
np.random.seed(42)

# Simulating participants
num_participants = 10

true_means = np.random.uniform(50, 150, num_participants)  # True means of participants

initial_guess = 100 # Prior guess for every participant

est_means = np.full(num_participants, initial_guess) #array of these guesses which will be altered iteratively

# Store participant data
participants = {
    i: {
        "true_mean": true_means[i],
        "estimated_mean": est_means[i],
    }
    for i in range(num_participants)
}

In [344]:
stddevs = np.full(num_participants, 5) # Fixed standard deviation of 5 for simplicity of model

### Update particpant information to include true reserve pricing

In [345]:
# Initialize ReservePriceCalculator with true means and stddevs
calculator = ReservePriceCalculator(true_means, stddevs)

# Calculate reserve prices for all participants
true_reserve_prices = calculator.calculate_reserve_prices()

# Update participant data with true reserve prices
for i, reserve_price in enumerate(true_reserve_prices):
    participants[i]["true_reserve_price"] = reserve_price

# Print the results
for i, data in participants.items() :
    print(f"Participant {i+1}: {data}")


Participant 1: {'true_mean': 87.45401188473625, 'estimated_mean': 100, 'true_reserve_price': 77.82702917193446}
Participant 2: {'true_mean': 145.07143064099162, 'estimated_mean': 100, 'true_reserve_price': 134.15087322892364}
Participant 3: {'true_mean': 123.1993941811405, 'estimated_mean': 100, 'true_reserve_price': 112.67800408478666}
Participant 4: {'true_mean': 109.86584841970367, 'estimated_mean': 100, 'true_reserve_price': 99.6342731799071}
Participant 5: {'true_mean': 65.60186404424365, 'estimated_mean': 100, 'true_reserve_price': 56.79570493836255}
Participant 6: {'true_mean': 65.59945203362027, 'estimated_mean': 100, 'true_reserve_price': 56.7934025369315}
Participant 7: {'true_mean': 55.80836121681995, 'estimated_mean': 100, 'true_reserve_price': 47.496608820983724}
Participant 8: {'true_mean': 136.61761457749353, 'estimated_mean': 100, 'true_reserve_price': 125.84183888351781}
Participant 9: {'true_mean': 110.11150117432088, 'estimated_mean': 100, 'true_reserve_price': 99.87

### Include auctionner guess of reserve price per candidate

In [346]:
# Initialize ReservePriceCalculator with estimated means and stddevs
est_calculator = ReservePriceCalculator(est_means, stddevs)

# Calculate reserve prices based on estimated means
est_reserve_prices = est_calculator.calculate_reserve_prices()

# Update participant data with estimated reserve prices
for i, est_reserve in enumerate(est_reserve_prices):
    participants[i]['Estimated_reserve_price'] = est_reserve

# Print the results
for i, data in participants.items():
    print(f"Participant {i+1}: {data}")
 

Participant 1: {'true_mean': 87.45401188473625, 'estimated_mean': 100, 'true_reserve_price': 77.82702917193446, 'Estimated_reserve_price': 90.01316311109564}
Participant 2: {'true_mean': 145.07143064099162, 'estimated_mean': 100, 'true_reserve_price': 134.15087322892364, 'Estimated_reserve_price': 90.01316311109564}
Participant 3: {'true_mean': 123.1993941811405, 'estimated_mean': 100, 'true_reserve_price': 112.67800408478666, 'Estimated_reserve_price': 90.01316311109564}
Participant 4: {'true_mean': 109.86584841970367, 'estimated_mean': 100, 'true_reserve_price': 99.6342731799071, 'Estimated_reserve_price': 90.01316311109564}
Participant 5: {'true_mean': 65.60186404424365, 'estimated_mean': 100, 'true_reserve_price': 56.79570493836255, 'Estimated_reserve_price': 90.01316311109564}
Participant 6: {'true_mean': 65.59945203362027, 'estimated_mean': 100, 'true_reserve_price': 56.7934025369315, 'Estimated_reserve_price': 90.01316311109564}
Participant 7: {'true_mean': 55.80836121681995, 'e

In [347]:
for i, data in participants.items():
    print(f"Particpant {i+1}: {data['true_reserve_price'],data['Estimated_reserve_price']}")

Particpant 1: (77.82702917193446, 90.01316311109564)
Particpant 2: (134.15087322892364, 90.01316311109564)
Particpant 3: (112.67800408478666, 90.01316311109564)
Particpant 4: (99.6342731799071, 90.01316311109564)
Particpant 5: (56.79570493836255, 90.01316311109564)
Particpant 6: (56.7934025369315, 90.01316311109564)
Particpant 7: (47.496608820983724, 90.01316311109564)
Particpant 8: (125.84183888351781, 90.01316311109564)
Particpant 9: (99.87419136028916, 90.01316311109564)
Particpant 10: (110.33486965151856, 90.01316311109564)


Eventually, we want to get the estimated reserve prices as close as possible to the true reserve pricings in order to optimsie our auction model.

### Payment using true reserve price

## changes need to be made, ie looking at the reserve price is being used, get a total utility function etc, may make a graph, framework there though.

### SGD probably not being used correctly, I like all the information being printed, inference not being done right?

In [353]:
class Auction:
    def __init__(self, num_participants, stddevs, true_means):
        self.num_participants = num_participants
        self.stddevs = stddevs  
        self.true_means = true_means
        self.est_means = np.full(num_participants, 100)  # Initial estimates

        # Initialize ReservePriceCalculator with true means and standard deviations to calculate true reserve prices
        self.true_reserve_price_calculator = ReservePriceCalculator(self.true_means, self.stddevs)
        self.true_reserve_prices = self.true_reserve_price_calculator.calculate_reserve_prices()

        # Initialize ReservePriceCalculator with estimated means and standard deviations to calculate estimated reserve prices
        self.reserve_price_calculator = ReservePriceCalculator(self.est_means, self.stddevs)

        # Store participants' data
        self.participants = {
            i: {
                'true_mean': true_means[i],
                'estimated_mean': self.est_means[i],
                'true_reserve_price': self.true_reserve_prices[i],  # Keep true reserve prices
                'Estimated_reserve_price': None,  # Will calculate this later based on estimated means
            }
            for i in range(num_participants)
        }

    def generate_bid(self, true_mean, stddev):
        """Generate a random bid based on true mean and standard deviation."""
        return norm.rvs(true_mean, stddev)

    def run_auction(self):
        """
        Run a single round of the auction.
        """
        # Generate bids for all participants
        bids = [self.generate_bid(mean, stddev) for mean, stddev in zip(self.true_means, self.stddevs)]

        # Calculate estimated reserve prices based on estimated means
        est_reserve_prices = self.reserve_price_calculator.calculate_reserve_prices()

        # Find the highest bid and second-highest bid
        sorted_bids = sorted(zip(bids, range(self.num_participants)), reverse=True)
        highest_bid, highest_idx = sorted_bids[0]
        second_highest_bid = sorted_bids[1][0]

        # Check if the highest virtual valuation exceeds the reserve price
        highest_virtual_valuation = self.reserve_price_calculator.virtual_valuation(
            highest_bid, self.est_means[highest_idx], self.stddevs[highest_idx]
        )
        payment = None
        if highest_virtual_valuation >= est_reserve_prices[highest_idx]:
            # Item is allocated to highest bidder, apply payment rule
            payment = max(second_highest_bid, est_reserve_prices[highest_idx])

        # Update all participants' estimated means using their bids
        self.update_estimates(bids)

        # Recalculate the estimated reserve prices based on the updated estimated means
        self.reserve_price_calculator = ReservePriceCalculator(self.est_means, self.stddevs)
        est_reserve_prices = self.reserve_price_calculator.calculate_reserve_prices()

        # Update participant data with the recalculated estimated reserve prices
        for i, est_reserve in enumerate(est_reserve_prices):
            self.participants[i]['Estimated_reserve_price'] = est_reserve

        # Calculate profit using true reserve prices
        true_reserve_profit = self.calculate_profit_with_true_reserve_prices(bids)

        # Collect results for this round
        round_results = {
            "bids": bids,
            "true_reserve_prices": self.true_reserve_prices,
            "est_reserve_prices": est_reserve_prices,
            "highest_bid": highest_bid,
            "highest_idx": highest_idx,
            "second_highest_bid": second_highest_bid,
            "payment": payment,
            "updated_estimates": self.est_means.copy(),
            "true_reserve_profit": true_reserve_profit,  # Store profit using true reserve prices
            "est_reserve_profit": payment if payment is not None else 0,  # Profit using estimated reserve prices
        }

        # Print round results
        print(f"Round Results:")
        print(f"Bids: {bids}")
        print(f"True Reserve Prices: {self.true_reserve_prices}")
        print(f"Estimated Reserve Prices: {est_reserve_prices}")
        print(f"Highest Bid: {highest_bid} by Participant {highest_idx + 1}")
        print(f"Second Highest Bid: {second_highest_bid}")
        print(f"Payment: {payment}")
        print(f"Updated Estimates: {self.est_means}")
        print(f"True Reserve Profit: {true_reserve_profit}")
        print(f"Estimated Reserve Profit: {payment if payment is not None else 0}")
        print()

        return round_results

    def update_estimates(self, bids):
        """
        Use SGD to update the estimated mean for each participant based on their bids.
        """
        for idx, bid in enumerate(bids):
            # Use a simple learning rate (eta) for manual SGD updates
            learning_rate = 0.1
            self.est_means[idx] = (1 - learning_rate) * self.est_means[idx] + learning_rate * bid

    def calculate_profit_with_true_reserve_prices(self, bids):
        """Calculate the profit using the true reserve prices for a round."""
        profit = 0

        # Generate the profit based on true reserve prices and the bids
        sorted_bids = sorted(zip(bids, range(self.num_participants)), reverse=True)
        highest_bid, highest_idx = sorted_bids[0]
        second_highest_bid = sorted_bids[1][0]

        # Check if the highest bid exceeds the true reserve price
        if highest_bid >= self.true_reserve_prices[highest_idx]:
            # Apply payment rule using the true reserve price
            payment = max(second_highest_bid, self.true_reserve_prices[highest_idx])
            profit += payment

        return profit

    def calculate_total_profit(self, num_rounds):
        """Calculate the total profit (sum of payments) over multiple rounds."""
        total_true_reserve_profit = 0
        total_est_reserve_profit = 0
        for _ in range(num_rounds):
            round_results = self.run_auction()  # Run the auction for a round
            total_true_reserve_profit += round_results["true_reserve_profit"]
            total_est_reserve_profit += round_results["est_reserve_profit"]
        return total_true_reserve_profit, total_est_reserve_profit


# Example usage
np.random.seed(42)
num_participants = 10
true_means = np.random.uniform(50, 150, num_participants)  # True means of participants
stddevs = np.full(num_participants, 5)  # Standard deviations for each participant
num_rounds = 100  # Number of rounds to simulate

# Initialize the auction with both true and estimated means
auction = Auction(num_participants, stddevs, true_means)

# Calculate the total profit for the auction over multiple rounds (both true and estimated reserve profits)
total_true_reserve_profit, total_est_reserve_profit = auction.calculate_total_profit(num_rounds)

print(f"Total Profit using True Reserve Prices over {num_rounds} rounds: {total_true_reserve_profit}")
print(f"Total Profit using Estimated Reserve Prices over {num_rounds} rounds: {total_est_reserve_profit}")

Round Results:
Bids: [85.10663995506148, 147.78423085892143, 120.8823057170782, 107.53719965185238, 66.81167540207383, 56.033050810331275, 47.18377205425478, 133.80617693128866, 105.04734557264877, 122.37849444258093]
True Reserve Prices: [77.82702917193446, 134.15087322892364, 112.67800408478666, 99.6342731799071, 56.79570493836255, 56.7934025369315, 47.496608820983724, 125.84183888351781, 99.87419136028916, 110.33486965151856]
Estimated Reserve Prices: [88.06653537327269, 93.91038811293667, 91.9611343614835, 90.01316311109564, 86.12131642953085, 85.14925692003817, 84.17757622741432, 92.93560458097649, 90.01316311109564, 91.9611343614835]
Highest Bid: 147.78423085892143 by Participant 2
Second Highest Bid: 133.80617693128866
Payment: 133.80617693128866
Updated Estimates: [ 98 104 102 100  96  95  94 103 100 102]
True Reserve Profit: 134.15087322892364
Estimated Reserve Profit: 133.80617693128866

Round Results:
Bids: [82.91389150713019, 138.00991213431516, 130.52763802574827, 108.7369

Problem: second highest bid is always higher than reserve price, resulting in no updates, might have to go back to iteravilty updating the means based on bids. Or, go more explotative, and try extract higher utility by setting a higher reserve price which is higher than second highest bid usually, has it trade offs where we might have rounds where we extract 0 profit but end up with more. If we can somehow set it up so that the reserve pricing updates based on average second highest price whilst taking into account the individual average means of each particpant? 